Import the database

In [1]:
import sqlite3
import pandas as pd
import os

db_path = os.path.join('..', 'database', 'smite_players.db')
conn = sqlite3.connect(db_path)
c = conn.cursor()

# Execute a query to retrieve all records
c.execute("SELECT * FROM combined_data5")

# Get the column names
column_names = [description[0] for description in c.description]

# Fetch all the results
rows = c.fetchall()

# Check if there are any results
print(f"Number of records retrieved: {len(rows)}")

# Convert the results into a pandas DataFrame
if rows:
    df = pd.DataFrame(rows, columns=column_names)
    print("DataFrame created successfully.")
else:
    print("No records found.")


Número de registros obtenidos: 262314
DataFrame creado exitosamente.


## Trying differents models

Preprocessing

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping

# Function to split and ensure each list has 3 elements
def split_tags(tags):
    split = tags.split(', ') if pd.notna(tags) else [None, None, None]
    while len(split) < 3:
        split.append(None)
    return split[:3]

# Step 1: Split the tag columns into 3 components
df[['character_tag_1', 'character_tag_2', 'character_tag_3']] = pd.DataFrame(df['character_tags'].apply(split_tags).tolist(), index=df.index)
df[['enemy_1_tag_1', 'enemy_1_tag_2', 'enemy_1_tag_3']] = pd.DataFrame(df['enemy_1_tags'].apply(split_tags).tolist(), index=df.index)
df[['enemy_2_tag_1', 'enemy_2_tag_2', 'enemy_2_tag_3']] = pd.DataFrame(df['enemy_2_tags'].apply(split_tags).tolist(), index=df.index)
df[['enemy_3_tag_1', 'enemy_3_tag_2', 'enemy_3_tag_3']] = pd.DataFrame(df['enemy_3_tags'].apply(split_tags).tolist(), index=df.index)

# Step 2: Encode categorical variables
categorical_columns = [
    'character_name', 'enemy_1_character', 'enemy_2_character', 'enemy_3_character',
    'character_class_distance', 'character_type_dmg', 'character_type_dmgform',
    'character_tag_1', 'character_tag_2', 'character_tag_3',  
    'enemy_1_class_distance', 'enemy_1_type_dmg', 'enemy_1_type_dmgform',
    'enemy_1_tag_1', 'enemy_1_tag_2', 'enemy_1_tag_3',  
    'enemy_2_class_distance', 'enemy_2_type_dmg', 'enemy_2_type_dmgform',
    'enemy_2_tag_1', 'enemy_2_tag_2', 'enemy_2_tag_3',  
    'enemy_3_class_distance', 'enemy_3_type_dmg', 'enemy_3_type_dmgform',
    'enemy_3_tag_1', 'enemy_3_tag_2', 'enemy_3_tag_3'
]

# Apply One-Hot Encoding
df_encoded = pd.get_dummies(df, columns=categorical_columns, drop_first=True)

# Read processed items from file
with open("items_procesados.txt", "r", encoding="utf-8") as file:
    items_procesados = [line.strip() for line in file]

# Step 1: Extract all unique items
all_items = df['build'].str.split(', ').explode().unique()

# Create binary columns only for permitted items
for item in all_items:
    df[item] = df['build'].apply(lambda x: 1 if item in x.split(', ') else 0)

# Step 2: Define predictor variables (X) and target variable (y)
X = df_encoded[df_encoded.columns[df_encoded.columns.str.startswith(tuple(categorical_columns))]]
df['damage'] = pd.to_numeric(df['damage'].str.replace(',', ''), errors='coerce')
df['damage_taken'] = pd.to_numeric(df['damage_taken'].str.replace(',', ''), errors='coerce')
df['damage_mitigated'] = pd.to_numeric(df['damage_mitigated'].str.replace(',', ''), errors='coerce')

# Calculate build score
df['build_score'] = df['damage']

build_columns = df.columns[df.columns.isin(all_items)]

# Add build columns to predictor variables X
X = pd.concat([X, df[build_columns]], axis=1)

C:\Users\joseb\AppData\Local\Temp\ipykernel_17380\520493886.py:48: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[item] = df['build'].apply(lambda x: 1 if item in x.split(', ') else 0)
C:\Users\joseb\AppData\Local\Temp\ipykernel_17380\520493886.py:48: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[item] = df['build'].apply(lambda x: 1 if item in x.split(', ') else 0)
C:\Users\joseb\AppData\Local\Temp\ipykernel_17380\520493886.py:48: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `f

Training

NN

In [62]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping
import tensorflow as tf

# Define a custom loss function for RMSE using TensorFlow backend
def rmse(y_true, y_pred):
    return tf.keras.backend.sqrt(tf.keras.backend.mean(tf.keras.backend.square(y_pred - y_true)))

# Assuming that the build encoding and the dataset X and the target variable 'build_score' are already prepared

# Split into training and test set (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, df['build_score'], test_size=0.2, random_state=42)

# Feature scaling (recommended for neural networks)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Build the Neural Network model
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))  # Input layer
model.add(BatchNormalization())  # Batch Normalization
model.add(Dropout(0.5))  # Dropout to avoid overfitting
model.add(Dense(64, activation='relu'))  # Second dense layer
model.add(Dense(1, activation='linear'))  # Output layer (for regression)

# Compile the model
model.compile(optimizer='adam', loss=rmse)

# Define early stopping (to stop training if no improvement)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=32, callbacks=[early_stopping])

# Evaluate the performance on the test set
test_loss = model.evaluate(X_test, y_test)
print(f"Test Loss (RMSE): {test_loss}")

# Continue training the model with the test set
model.fit(X_test, y_test, epochs=10, batch_size=32, verbose=1)


c:\Users\joseb\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
6558/6558 ━━━━━━━━━━━━━━━━━━━━ 19s 3ms/step - loss: 9668.4414 - val_loss: 5182.5298
Epoch 2/100
6558/6558 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - loss: 5507.1797 - val_loss: 5060.5449
Epoch 3/100
6558/6558 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - loss: 5412.6982 - val_loss: 5015.3843
Epoch 4/100
6558/6558 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - loss: 5387.4736 - val_loss: 4996.8696
Epoch 5/100
6558/6558 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - loss: 5293.3960 - val_loss: 4966.2363
Epoch 6/100
6558/6558 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - loss: 5303.4136 - val_loss: 4971.4683
Epoch 7/100
6558/6558 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - loss: 5292.2153 - val_loss: 4960.6138
Epoch 8/100
6558/6558 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - loss: 5239.3477 - val_loss: 4956.5552
Epoch 9/100
6558/6558 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - loss: 5221.4512 - val_loss: 5006.1143
Epoch 10/100
6558/6558 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - loss: 5235.0557 - val_loss: 4929.7549
Epoch 11/100
6558/6558 ━━━━━━━━━━━━━━━━

In [34]:
# Save the trained model
model.save('nn_build_score_model.h5')

CatBoosting

In [5]:
import optuna
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, df['build_score'], test_size=0.2, random_state=42)

# Further split the training set into training and validation sets
X_train_train, X_train_validation, y_train_train, y_train_validation = train_test_split(X_train, y_train, test_size=0.25, random_state=42)  # 25% of 80% is 20% of the total

# Define the objective function for Optuna
def objective(trial):
    # Suggest values for the hyperparameters
    iterations = trial.suggest_int('iterations', 500, 2000)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-3, 0.1)
    depth = trial.suggest_int('depth', 4, 10)
    l2_leaf_reg = trial.suggest_float('l2_leaf_reg', 1.0, 10.0)
    bagging_temperature = trial.suggest_float('bagging_temperature', 0.0, 1.0)
    
    # Create the CatBoost model with the suggested hyperparameters
    catboost_model = CatBoostRegressor(
        iterations=iterations,
        learning_rate=learning_rate,
        depth=depth,
        l2_leaf_reg=l2_leaf_reg,
        bagging_temperature=bagging_temperature,
        random_state=42,
        verbose=0
    )
    
    # Train the model on the training set
    catboost_model.fit(X_train_train, y_train_train)
    
    # Make predictions on the validation set
    y_pred_val = catboost_model.predict(X_train_validation)
    
    # Calculate the RMSE on the validation set
    rmse = mean_squared_error(y_train_validation, y_pred_val, squared=False)
    
    return rmse

# Create an Optuna study to minimize RMSE
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

# Retrieve the best hyperparameters
best_params = study.best_params
print(f"Best hyperparameters: {best_params}")

# Train the model with the best hyperparameters on the original full training set
best_catboost_model = CatBoostRegressor(**best_params, random_state=42, verbose=0)
best_catboost_model.fit(X_train, y_train)

# Make predictions with the best model on the test set
y_pred_best = best_catboost_model.predict(X_test)

# Evaluate the model's performance on the test set
rmse_best = mean_squared_error(y_test, y_pred_best, squared=False)
print(f"Root Mean Squared Error CatBoost (tuned model): {rmse_best}")



[I 2024-10-13 15:57:09,115] A new study created in memory with name: no-name-2aa2ac5c-53dd-475e-b73a-a7c23d7b9316
C:\Users\joseb\AppData\Local\Temp\ipykernel_17380\23150180.py:16: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-3, 0.1)
c:\Users\joseb\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
[I 2024-10-13 15:57:26,101] Trial 0 finished with value: 6348.636099331196 and parameters: {'iterations': 688, 'learning_rate': 0.004362539893743521, 'depth': 5, 'l2_leaf_reg': 3.6785580176922448, 'bagging_temperature': 0.04988310999057444}. Best is trial 0 with value: 6348.636099

Mejores hiperparámetros: {'iterations': 1983, 'learning_rate': 0.09434483707156406, 'depth': 10, 'l2_leaf_reg': 2.542819560694613, 'bagging_temperature': 0.23798239433578128}
Root Mean Squared Error CatBoost (modelo ajustado): 5057.267510624712


c:\Users\joseb\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Random Forest

In [8]:
import optuna
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, df['build_score'], test_size=0.2, random_state=42)

# Further split the training set into training and validation sets
X_train_train, X_train_validation, y_train_train, y_train_validation = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

# Define the objective function for Optuna
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 100, 1000)
    max_depth = trial.suggest_int('max_depth', 4, 30)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
    max_features = trial.suggest_categorical('max_features', ['sqrt', 'log2', None])  # Adjustment here

    # Create the RandomForest model with the suggested hyperparameters
    rf_model = RandomForestRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        random_state=42
    )
    
    # Train the model on the training set
    rf_model.fit(X_train_train, y_train_train)
    
    # Make predictions on the validation set
    y_pred_val = rf_model.predict(X_train_validation)
    
    # Calculate RMSE on the validation set
    rmse = mean_squared_error(y_train_validation, y_pred_val, squared=False)
    
    return rmse

# Create an Optuna study for minimizing RMSE
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

# Retrieve the best hyperparameters
best_params = study.best_params
print(f"Best hyperparameters: {best_params}")

# Train the model with the best hyperparameters on the full original training set
best_rf_model = RandomForestRegressor(**best_params, random_state=42)
best_rf_model.fit(X_train, y_train)

# Make predictions with the best model on the test set
y_pred_best = best_rf_model.predict(X_test)

# Evaluate the model
rmse_best = mean_squared_error(y_test, y_pred_best, squared=False)
print(f"Root Mean Squared Error Random Forest (tuned model): {rmse_best}")



[I 2024-10-13 19:12:56,645] A new study created in memory with name: no-name-9bd57ba5-0aed-4a53-9663-7f4d1c04004e
c:\Users\joseb\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
[I 2024-10-13 19:15:00,543] Trial 0 finished with value: 7313.929923720392 and parameters: {'n_estimators': 884, 'max_depth': 20, 'min_samples_split': 5, 'min_samples_leaf': 8, 'max_features': 'log2'}. Best is trial 0 with value: 7313.929923720392.
c:\Users\joseb\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
[I 2024-10-13 19:15:36,158] Trial 1 finished with value: 7508.545435191777 and parameters: {'n_estimators': 306

KeyboardInterrupt: 